In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)
n_rows = 2000
categories = ['Industrial Parts', 'Electronic Components', 'Office Supplies', 'Safety Equipment']
regions = ['North America', 'Europe', 'APAC', 'LAPAM']
customers = [f'CUST-{i:04d}' for i in range(1, 301)]
data = {
    'Order_ID': [f'ORD-{i:05d}' for i in range(1, n_rows + 1)],
    'Order_Date': [datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365)) for _ in range(n_rows)],
    'Customer_ID': [np.random.choice(customers) for _ in range(n_rows)],
    'Category': [np.random.choice(categories) for _ in range(n_rows)],
    'Region': [np.random.choice(regions) for _ in range(n_rows)],
    'Sales': np.random.uniform(50, 5000, n_rows).round(2),
}
df = pd.DataFrame(data)
margin_map = {
    'Industrial Parts': 0.25,
    'Electronic Components': 0.15,
    'Office Supplies': 0.05,
    'Safety Equipment': 0.12
}
df['Unit_Cost'] = df.apply(
    lambda x: x['Sales'] * (1 - margin_map[x['Category']] + np.random.uniform(-0.05, 0.05)),
    axis=1
).round(2)
df['Profit'] = (df['Sales'] - df['Unit_Cost']).round(2)
file_name = "ECommerce_Business_Data_2025.xlsx"
df.to_excel(file_name, index=False)
print(f"--- Dataset successfully generated: {file_name} ---")
print("Preview of the generated data:")
print(df.head())

--- Dataset successfully generated: ECommerce_Business_Data_2025.xlsx ---
Preview of the generated data:
    Order_ID Order_Date Customer_ID  ...    Sales Unit_Cost   Profit
0  ORD-00001 2024-04-12   CUST-0156  ...  4666.21   4258.52   407.69
1  ORD-00002 2024-12-14   CUST-0207  ...  2899.54   2593.86   305.68
2  ORD-00003 2024-09-27   CUST-0108  ...   674.27    548.36   125.91
3  ORD-00004 2024-04-16   CUST-0251  ...  2929.52   2277.61   651.91
4  ORD-00005 2024-03-12   CUST-0130  ...  3843.42   2792.17  1051.25

[5 rows x 8 columns]


In [5]:
import pandas as pd
import sqlite3
conn = sqlite3.connect(':memory:')
df.to_sql('sales_data', conn, index=False, if_exists='replace')
query = """
SELECT 
    Customer_ID,
    Order_Date,
    Category,
    Sales,
    SUM(Sales) OVER (PARTITION BY Customer_ID ORDER BY Order_Date) as Cumulative_Sales,
    LAG(Sales) OVER (PARTITION BY Customer_ID ORDER BY Order_Date) as Last_Order_Value,
    RANK() OVER (PARTITION BY Category ORDER BY Sales DESC) as Category_Sales_Rank
FROM 
    sales_data
ORDER BY 
    Customer_ID, Order_Date;
"""
sql_analysis = pd.read_sql_query(query, conn)
print("--- SQL Window Functions results successfully generated ---")
print(sql_analysis.head(10))
sql_analysis.to_excel("ECommerce_SQL_Analysis_Final.xlsx", index=False)

--- SQL Window Functions results successfully generated ---
  Customer_ID                 Order_Date  ... Last_Order_Value  Category_Sales_Rank
0   CUST-0108  2024-09-27 00:00:00+00:00  ...             None                    2
1   CUST-0130  2024-03-12 00:00:00+00:00  ...             None                    1
2   CUST-0156  2024-04-12 00:00:00+00:00  ...             None                    1
3   CUST-0207  2024-12-14 00:00:00+00:00  ...             None                    1
4   CUST-0251  2024-04-16 00:00:00+00:00  ...             None                    2

[5 rows x 7 columns]
